In [9]:
import qsharp
import qsharp_widgets
import json
from diskcache import Cache
import re_utils

re_utils.DEBUG = True

cache = Cache("~/quant-arith-cache/re-multipliers")
qsharp.init(project_root="../")

@cache.memoize()
def estimate_resources_constant_adder_pareto(op, n):
    est = qsharp.estimate(f"EstimateUtils.RunConstantAdder({n},{op})", params={"estimateType": "frontier"})
    return est    

ops_and_max_n = [
  ("Std.Arithmetic.IncByLUsingIncByLE(Std.Arithmetic.RippleCarryTTKIncByLE,_,_)", "TTK", 2**20),
  ("Std.Arithmetic.IncByLUsingIncByLE(Std.Arithmetic.RippleCarryCGIncByLE,_,_)", "Gidney", 2**20),
  ("Std.Arithmetic.IncByLUsingIncByLE(QuantumArithmetic.JHHA2016.Add_Mod2N,_,_)", "JHHA", 2**20),
  ("Std.Arithmetic.IncByLUsingIncByLE(QuantumArithmetic.CDKM2004.Add,_,_)", "CDKM", 2**20),
  ("Std.Arithmetic.IncByLUsingIncByLE(QuantumArithmetic.DKRS2004.Add,_,_)", "DKRS", 2**20),
  ("QuantumArithmetic.ConstAdder.AddConstant", "ConstAdder", 2**20),      
  ("QuantumArithmetic.PG2012.AddConstantQFT", "QFT", 27554),
]

In [10]:
n = 2**11
results = {} 
i = 0 
for op, _, _ in ops_and_max_n:
    results[i] = estimate_resources_constant_adder_pareto(op, n)
    i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, _ in ops_and_max_n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…

The test above was limited to `n=2**11` due to the QFT based adder. However, below is a plot of the results of `n=2**20` without the QFT constant adder.

In [6]:
n = 2**20
results = {}
i = 0
for op, _, max_n in ops_and_max_n:
    if n <= max_n:
        results[i] =  estimate_resources_constant_adder_pareto(op, n)
        i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, max_n in ops_and_max_n if max_n >= n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…

Since TTK, JHHA, CDKM, and ConstAdder overlap, below is a plot of the pareto frontier with `n=2**3` in order to show the differences.

In [14]:
closer_results = {}
n = 2**3
closer_results[0] = estimate_resources_constant_adder_pareto("Std.Arithmetic.IncByLUsingIncByLE(Std.Arithmetic.RippleCarryTTKIncByLE,_,_)", n)
closer_results[1] = estimate_resources_constant_adder_pareto("Std.Arithmetic.IncByLUsingIncByLE(QuantumArithmetic.JHHA2016.Add_Mod2N,_,_)",n)
closer_results[2] = estimate_resources_constant_adder_pareto("Std.Arithmetic.IncByLUsingIncByLE(QuantumArithmetic.CDKM2004.Add,_,_)", n)
closer_results[3] = estimate_resources_constant_adder_pareto("QuantumArithmetic.ConstAdder.AddConstant", n)

qsharp_widgets.EstimatesOverview(closer_results, runNames=["TTK", "JHHA", "CDKM", "ConstAdder"])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…